In [7]:
# conda install git

In [8]:
# pip install --user --upgrade git+https://github.com/twintproject/twint.git@origin/master#egg=twint

In [9]:
import numpy as np
import pandas as pd
import twint
import time
from datetime import datetime
from datetime import timedelta, date

In [10]:
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 199)
pd.options.display.float_format = '{:.2f}'.format

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [11]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import nltk
from nltk.tokenize import ToktokTokenizer
toktok = ToktokTokenizer()
from nltk.corpus import stopwords
# lemma = nltk.wordnet.WordNetLemmatizer()

In [ ]:
nltk.download('stopwords')

In [ ]:
stopwords = set(stopwords.words("english"))
stopwords -= {'who'}
stopwords.update(['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}']) # remove it if you need punctuation 

In [ ]:
def remove_junk_from_str(string):
    return " ".join([x for x in toktok.tokenize(string) if x.lower() not in stopwords and x[0]!='#' and x[:3]!='htt'])

vec_remove_junk = np.vectorize(remove_junk_from_str)

In [15]:
def clean_tweets(df):
    local_df = df
    
    #drop redundant cols
    local_df.drop(['created_at', 'timezone', 'language', 'cashtags', 'user_id_str', 'urls', 'photos', 'video', 'thumbnail', 'translate', 'trans_src', 'trans_dest', 'name', 'search'], axis=1, inplace=True)
    
    #convert date to datetime
    local_df['date'] = pd.to_datetime(local_df['date'])
    
    #REMOVE STOPWORDS FROM TWEET HERE
    local_df['tweet'] = list(map(vec_remove_junk, local_df['tweet']))
    
    return local_df


## Collect Tweets hashtags

## Fetch tweets within a date range

In [ ]:
def find_tweets_in_date_range(search_terms, start_date, end_date):
    '''    
    search_terms = list of strings (or singleton) of search terms that will be in the tweet
    
    returns a pandas dataframe of tweets
    '''
    
    search_string = " OR ".join(search_terms)
    
    c = twint.Config()
    
#     date range of search
    c.Since = start_date
    c.Until = end_date
    
    c.Search = search_string
    
    # only collect a certain number of tweets
#     c.Limit = num_tweets

    # Don't print output
    c.Hide_output = True
    
    # find shadow-banned accounts too - this apparently slows things down considerably
    c.Profile_full = True

#     c.Output = f"{num_tweets} tweets - {start_date} to {end_date}.csv"
    c.Pandas = True

    twint.run.Search(c)

#     tweets_df = twint.storage.panda.Tweets_df
    
#     return clean_tweets(tweets_df)

    clean_tweets(twint.storage.panda.Tweets_df).to_csv(fr"C:/Users/crackcocaine69xxx/Python Stuff/594/Twint/Twint Output/{start_date} to {end_date}.csv", index=False)

### Fetch tweets up to a number limit

In [16]:
def find_tweets_by_limit(num_tweets, search_terms):#, start_date, end_date):
    '''    
    search_terms = list of strings (or singleton) of search terms that will be in the tweet
    
    returns a pandas dataframe
    '''
    
    search_string = " OR ".join(search_terms)
    
    c = twint.Config()
    
    # date range of search
#     c.Since = start_date
#     c.Until = end_date
    
    c.Search = search_string
    
    # only collect a certain number of tweets
    c.Limit = num_tweets

    # Don't print output
    c.Hide_output = True
    
    # find shadow-banned accounts too - this apparently slows things down considerably
    c.Profile_full = True

#     c.Output = f"{num_tweets} tweets - {start_date} to {end_date}.csv"
    c.Pandas = True

    twint.run.Search(c)

    tweets_df = twint.storage.panda.Tweets_df
    
    return clean_tweets(tweets_df)

## Find most common associated hashtags

In [ ]:
def find_associated_hashtags(series):
    '''
    series = pd series where each observation is a python list
    '''
    hashtag_set = set([tag for list_of_tags in series for tag in list_of_tags])
    
    
    count_dict = dict.fromkeys(hashtag_set, 0)
    for list_of_tags in series:
        for tag in list_of_tags:
            count_dict[tag] += 1

    count_dict = dict(sorted(count_dict.items(), key=lambda item: item[1], reverse=True))

    
    return(count_dict)


## List of Hashtags to Search

In [ ]:
# conspiracy_hashtags is a list of hashtags that I think are used by people who don't believe that covid is real (or who belive that it's overblown).
# Note that there are conspiracy theorists who believe it does exist but was intentionally planted by china etc. these are not the conspiracy theorists I want!!!

#SHOULD I TRY TO SEPARATE THE TWO GROUPS? THEY MIGHT NOT EVEN BE SEPARATE???

general_conspiracy_hashtags = [
    'plandemic',
    'scamdemic',
    'covidhoax',
    'nwo',
    'covid1984',
    'plandemia',
    'agenda21',
    'thegreatreset',
    'agenda2030',
    'newworldorder',
    'wakeupamerica',
    'wakeup',
    'openamericanow',
    'firefauci',
    'wwg1wga',
    'qanon',
    'coronahoax'
]

general_conspiracy_hashtags = general_conspiracy_hashtags + [f'#{x}' for x in general_conspiracy_hashtags]

# covid_denier_hashtags = [
#     'plandemic',
#     'scamdemic',
#     'covidhoax',    
# ]

## Loop to find all tweets

### Create Date Ranges

In [35]:
week = timedelta(weeks=1)
sixdays = timedelta(days=6)

In [36]:
date_ranges = []

today_str = date.today().strftime('%d-%m-%Y')

for date in pd.date_range(start='8-15-2020', end=today_str, freq='1W'):
    date_tuple = ((date.strftime('%Y-%m-%d'), (date+sixdays).strftime('%Y-%m-%d') ))
    
    print(date_tuple, '\n')
    
    date_ranges.append(date_tuple)
    
# date_ranges    

('2020-08-16', '2020-08-22') 

('2020-08-23', '2020-08-29') 

('2020-08-30', '2020-09-05') 

('2020-09-06', '2020-09-12') 

('2020-09-13', '2020-09-19') 

('2020-09-20', '2020-09-26') 

('2020-09-27', '2020-10-03') 

('2020-10-04', '2020-10-10') 

('2020-10-11', '2020-10-17') 

('2020-10-18', '2020-10-24') 

('2020-10-25', '2020-10-31') 

('2020-11-01', '2020-11-07') 

('2020-11-08', '2020-11-14') 

('2020-11-15', '2020-11-21') 

('2020-11-22', '2020-11-28') 

('2020-11-29', '2020-12-05') 

('2020-12-06', '2020-12-12') 

('2020-12-13', '2020-12-19') 

('2020-12-20', '2020-12-26') 

('2020-12-27', '2021-01-02') 

('2021-01-03', '2021-01-09') 

('2021-01-10', '2021-01-16') 

('2021-01-17', '2021-01-23') 

('2021-01-24', '2021-01-30') 

('2021-01-31', '2021-02-06') 

('2021-02-07', '2021-02-13') 

('2021-02-14', '2021-02-20') 

('2021-02-21', '2021-02-27') 

('2021-02-28', '2021-03-06') 

('2021-03-07', '2021-03-13') 

('2021-03-14', '2021-03-20') 

('2021-03-21', '2021-03-27') 

('2021-0

## Final Loop

In [37]:
for date_tuple in date_ranges:
    find_tweets_in_date_range(general_conspiracy_hashtags, date_tuple[0], date_tuple[1])

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 1.0 secs


[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping w

CRITICAL:root:twint.run:Twint:Feed:noData'globalObjects'
sleeping for 1.0 secs


[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping w

In [155]:
mask_tweets_100k = find_tweets_by_limit(100000, ['mask'])

## Trash

### Make "general_conspiracy_400k"

In [166]:
general_conspiracy_400k = find_tweets_by_limit(400000, general_conspiracy_hashtags)

In [172]:
general_conspiracy_400k.to_csv(r"400k Conspiracy Tweets.csv", index=False)

In [167]:
find_associated_hashtags(general_conspiracy_400k['hashtags'])

{'wakeup': 85607,
 'agenda2030': 53037,
 'plandemia': 51217,
 'thegreatreset': 47530,
 'nwo': 43913,
 'wakeupamerica': 37713,
 'plandemic': 28670,
 'scamdemic': 27545,
 'covid19': 26953,
 'covid1984': 26680,
 'newworldorder': 17401,
 'agenda21': 13270,
 'greatreset': 11285,
 'covid': 10442,
 'immusic': 9620,
 'buildbackbetter': 9353,
 'covidhoax': 8718,
 'ods': 6956,
 'wef': 6822,
 'coronavirus': 6340,
 'truth': 6018,
 'maga': 4915,
 'lockdown': 4546,
 'freedom': 3967,
 'vaccine': 3857,
 'covid_19': 3760,
 'yonomevacuno': 3479,
 'klausschwab': 3275,
 'sdgs': 3116,
 'coronatimo': 3103,
 'votefraud': 2940,
 'america': 2874,
 'trump': 2866,
 'noalbozal': 2710,
 'usa': 2680,
 'billgates': 2447,
 'news': 2282,
 'crimesagainsthumanity': 2249,
 'corona': 2239,
 'enoughisenough': 2220,
 'covidvaccine': 2216,
 'noalavacuna': 2162,
 'elections': 2081,
 'china': 2071,
 'electionresults2020': 2057,
 'thefirstontv': 2037,
 'plandemie': 1999,
 'genocidio': 1995,
 'biden': 1992,
 'prepper': 1978,
 'l

### Find tweets associated with fauci

In [ ]:
find_tweets_by_limit(100, ['fauci, #firefauci'])

In [190]:
thegreatawakening_10k = find_tweets_by_limit(10000, ['#thegreatawakening'])

In [191]:
find_associated_hashtags(thegreatawakening_10k['hashtags'])

{'thegreatawakening': 10011,
 'wwg1wga': 1679,
 'maga': 1010,
 'thegreatreset': 819,
 'godwins': 677,
 'qanon': 626,
 'deepstate': 559,
 'kag': 533,
 'wakeupamerica': 488,
 'trump2020': 487,
 'tyranny': 463,
 'covid19': 415,
 'fightback': 411,
 'trump': 341,
 'stopthecoup': 340,
 'america': 335,
 'wearethenewsnow': 332,
 'darktolight': 320,
 'savethechildren': 305,
 'q': 302,
 'fakenews': 291,
 'thestorm': 288,
 'china': 275,
 'saveamerica': 272,
 'freedom': 271,
 'obamagate': 268,
 'tryants': 253,
 'greatawakening': 252,
 'vaccines': 248,
 'ccpvirus': 246,
 'uprising': 246,
 'lockdown': 243,
 'pushback': 237,
 'virus': 236,
 'draintheswamp': 235,
 'sheepnomore': 235,
 'wakeup': 227,
 'civildisobedience': 222,
 'treason': 222,
 'truth': 208,
 'kolr10news': 206,
 'ky3': 205,
 'kolr10': 200,
 'stopthesteal': 194,
 'ozarks': 193,
 'ky3news': 193,
 'ozarkmo': 191,
 'ksprnews': 190,
 'shadowgate': 189,
 'kspr': 189,
 'ozarksfox': 186,
 'thegreatawakeningworldwide': 182,
 'wethepeople': 179,

In [ ]:
find_associated_hashtags(fauci_100k['hashtags'])

In [17]:
ssc_tagged = find_tweets_by_limit(['#ssc'], 10000)

TypeError: can only join an iterable